In [3]:
import pandas as pd
import seaborn as sns
import networkx as nx
import numpy as np
import re
import matplotlib.pyplot as plt
import itertools
import sklearn.datasets
from sklearn.metrics.pairwise import euclidean_distances
import math
from sklearn.metrics.pairwise import rbf_kernel

In [4]:
data = sklearn.datasets.load_iris()
df = pd.DataFrame(data=data.data, columns=data.feature_names)

In [5]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [6]:
dfNumeric = df.select_dtypes(include=np.number)
dfNumeric

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [7]:
df.head(17)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


In [8]:
def euclideanDistance(source, target):
    totalSum = 0
    for i in range(len(source)):
        totalSum += pow(source[i] - target[i],2)
    return math.sqrt(totalSum)

def getDistanceMatrix(df):
    matrix = []
    for i in range(len(df)):
        matrix.append([])
    for i in range(len(df)):
        for j in range(len(df)):
            matrix[i].append(euclideanDistance(df.iloc[i],df.iloc[j]))
    return matrix


In [9]:
rbf_distances = rbf_kernel(data.data, gamma=1)
rbf_distances

array([[1.00000000e+00, 7.48263568e-01, 7.71051586e-01, ...,
        2.30082059e-09, 4.03840951e-10, 3.59908895e-08],
       [7.48263568e-01, 1.00000000e+00, 9.13931185e-01, ...,
        1.62136087e-09, 2.15082380e-10, 3.22418674e-08],
       [7.71051586e-01, 9.13931185e-01, 1.00000000e+00, ...,
        3.65410404e-10, 6.16221335e-11, 9.42405852e-09],
       ...,
       [2.30082059e-09, 1.62136087e-09, 3.65410404e-10, ...,
        1.00000000e+00, 6.83861409e-01, 6.63650250e-01],
       [4.03840951e-10, 2.15082380e-10, 6.16221335e-11, ...,
        6.83861409e-01, 1.00000000e+00, 5.54327285e-01],
       [3.59908895e-08, 3.22418674e-08, 9.42405852e-09, ...,
        6.63650250e-01, 5.54327285e-01, 1.00000000e+00]])

In [10]:
# Similarity matrix -> Has weird values, bit different, perhaps I need to make my own distance matrix
distances = np.array(getDistanceMatrix(df))
sigma = 1
distances = distances / -(2 * pow(sigma, 2))
distances = np.exp(distances)

In [ ]:
def kNNAlgorithm(k, distances):
    edgeList = {}
    for i in range(0, len(distances)):
        edgeList[i] = []
        for n in range(0, k):
            shortestDistance = -1
            shortestIndex = -1
            for j in range(0, len(distances)):
                if distances[i][j] >= shortestDistance and i != j: # I called it distance but its actually similarity, the higher the better
                    shortestDistance = distances[i][j]
                    shortestIndex = j
            distances[i][shortestIndex] = -1 # Set to inf distance thats already found to be closest
            if shortestIndex in edgeList.keys():
                if (i, shortestDistance) not in edgeList[shortestIndex]:
                    edgeList[i].append((shortestIndex, shortestDistance))
            else:
                edgeList[i].append((shortestIndex, shortestDistance))
    return edgeList

In [14]:
kNNResult = kNNAlgorithm(3, distances=np.copy(rbf_distances))
kNNResult

{0: [],
 1: [],
 2: [],
 3: [],
 4: [(0, 0.9801986733067592)],
 5: [],
 6: [(2, 0.9323938199059414)],
 7: [(0, 0.9704455335485139)],
 8: [(3, 0.9139311852712251)],
 9: [(1, 0.970445533548507)],
 10: [],
 11: [(7, 0.9512294245007167)],
 12: [(1, 0.9801986733067523), (9, 0.9704455335485002)],
 13: [(8, 0.8869204367171535)],
 14: [],
 15: [(14, 0.7408182206817252), (5, 0.6838614092123589)],
 16: [(10, 0.8869204367171598)],
 17: [(0, 0.99004983374917), (4, 0.9704455335485139)],
 18: [(5, 0.8958341352965224), (10, 0.860707976425059)],
 19: [],
 20: [],
 21: [(19, 0.9801986733067523), (17, 0.9417645335842466)],
 22: [(6, 0.8105842459701922), (2, 0.7710515858035624)],
 23: [],
 24: [(11, 0.9139311852712121)],
 25: [(9, 0.9607894391523104)],
 26: [(23, 0.9607894391523241), (7, 0.9512294245007167)],
 27: [(0, 0.9801986733067523)],
 28: [(0, 0.9801986733067592), (27, 0.9801986733067523)],
 29: [(3, 0.9704455335485002)],
 30: [(29, 0.9801986733067592), (9, 0.970445533548507)],
 31: [(20, 0.923116

In [15]:
def networkToDataframe(edgeList : dict):
    result = []
    i = 0
    for key, value in edgeList.items():
        for pair in value:
            result.append([])
            result[i].append(key)
            result[i].append(pair[0])
            result[i].append(pair[1])
            i += 1
    return result


In [16]:
def eRadius(epsilon, distances):
    edgeList = {}
    for i in range(0, len(distances)):
        edgeList[i] = []
    for i in range(0, len(distances)):
        for j in range(0, len(distances)):
            if distances[i][j] > epsilon and j not in edgeList[i] and i != j: # If similarity is higher than 0.9 (epsilon)? 
                edgeList[i].append((j, distances[i][j]))
    return edgeList

In [17]:
dfKNN = pd.DataFrame(data=networkToDataframe(edgeList=kNNResult), columns=['from', 'to', 'similarity'])

In [18]:
dfKNN.to_csv("KNNAlgorithmResult.csv")
display(dfKNN)

,from,to,similarity
0,4,0,0.980199
1,6,2,0.932394
2,7,0,0.970446
3,8,3,0.913931
4,9,1,0.970446
...,...,...,...
214,148,115,0.913931
215,148,110,0.733447
216,149,127,0.923116
217,149,138,0.904837


In [22]:
resultERadius = eRadius(0.9, np.copy(rbf_distances))
print(resultERadius)
dfERadius = pd.DataFrame(data=networkToDataframe(edgeList=resultERadius), columns=['from', 'to', 'similarity'])
display(dfERadius)
dfERadius.to_csv("eRadiusResult.csv")

{0: [(4, 0.9801986733067592), (7, 0.9704455335485139), (17, 0.99004983374917), (21, 0.9139311852712315), (26, 0.9048374180359583), (27, 0.9801986733067523), (28, 0.9801986733067592), (37, 0.9417645335842466), (39, 0.9801986733067523), (40, 0.9704455335485139), (48, 0.9139311852712251), (49, 0.9512294245007099)], 1: [(2, 0.9139311852712251), (9, 0.970445533548507), (12, 0.9801986733067523), (25, 0.9512294245007099), (30, 0.9417645335842466), (34, 0.9801986733067523), (35, 0.9139311852712251), (45, 0.9801986733067453), (49, 0.9048374180359519)], 2: [(1, 0.9139311852712251), (3, 0.9417645335842533), (6, 0.9323938199059414), (9, 0.9048374180359519), (12, 0.9323938199059479), (29, 0.9139311852712251), (34, 0.9139311852712315), (35, 0.9048374180359647), (42, 0.9139311852712315), (45, 0.9323938199059414), (47, 0.9801986733067523)], 3: [(2, 0.9417645335842533), (8, 0.9139311852712251), (9, 0.9048374180359583), (12, 0.9323938199059546), (29, 0.9704455335485002), (30, 0.9512294245007099), (34, 0

,from,to,similarity
0,0,4,0.980199
1,0,7,0.970446
2,0,17,0.990050
3,0,21,0.913931
4,0,26,0.904837
...,...,...,...
477,147,110,0.951229
478,148,115,0.913931
479,148,136,0.941765
480,149,127,0.923116


In [23]:
def kNNERadius(k, epsilon, distances):
    edgeList = {}
    for i in range(0, len(distances)):
        edgeList[i] = []
        for n in range(0, k):
            shortestDistance = -1
            shortestIndex = -1
            for j in range(0, len(distances)):
                if distances[i][j] >= shortestDistance and i != j and distances[i][j] > epsilon: # I called it distance but its actually similarity, the higher the better
                    shortestDistance = distances[i][j]
                    shortestIndex = j
            distances[i][shortestIndex] = -1 # Set to inf distance thats already found to be closest
            if shortestIndex != -1:
                edgeList[i].append((shortestIndex, shortestDistance))
    return edgeList

In [24]:
resultCombi = kNNERadius(3, 0.9, np.copy(rbf_distances))
resultCombi

{0: [(17, 0.99004983374917),
  (28, 0.9801986733067592),
  (4, 0.9801986733067592)],
 1: [(34, 0.9801986733067523),
  (12, 0.9801986733067523),
  (45, 0.9801986733067453)],
 2: [(47, 0.9801986733067523),
  (3, 0.9417645335842533),
  (12, 0.9323938199059479)],
 3: [(47, 0.9801986733067592),
  (29, 0.9704455335485002),
  (30, 0.9512294245007099)],
 4: [(37, 0.9801986733067592),
  (0, 0.9801986733067592),
  (40, 0.9704455335485139)],
 5: [],
 6: [(47, 0.9512294245007235),
  (2, 0.9323938199059414),
  (11, 0.9139311852712251)],
 7: [(39, 0.9900498337491771),
  (49, 0.9801986733067592),
  (0, 0.9704455335485139)],
 8: [(38, 0.9801986733067523),
  (3, 0.9139311852712251),
  (42, 0.9048374180359519)],
 9: [(34, 0.99004983374917), (30, 0.970445533548507), (1, 0.970445533548507)],
 10: [(48, 0.99004983374917),
  (27, 0.9231163463866373),
  (36, 0.9139311852712381)],
 11: [(7, 0.9512294245007167),
  (29, 0.9512294245007032),
  (26, 0.9231163463866373)],
 12: [(1, 0.9801986733067523),
  (9, 0.970

In [25]:
dfCombi = pd.DataFrame(data=networkToDataframe(edgeList=resultCombi), columns=['from', 'to', 'similarity'])
display(dfCombi)
dfCombi.to_csv("CombinationResult.csv")

,from,to,similarity
0,0,17,0.990050
1,0,28,0.980199
2,0,4,0.980199
3,1,34,0.980199
4,1,12,0.980199
...,...,...,...
260,147,110,0.951229
261,148,136,0.941765
262,148,115,0.913931
263,149,127,0.923116


In [26]:
distances

array([[1.        , 0.76394595, 0.77495449, ..., 0.10753807, 0.09774403,
        0.12618273],
       [0.76394595, 1.        , 0.86070798, ..., 0.1054578 , 0.09451231,
        0.12534868],
       [0.77495449, 0.86070798, 1.        , ..., 0.09722062, 0.08853515,
        0.11655191],
       ...,
       [0.10753807, 0.1054578 , 0.09722062, ..., 1.        , 0.73475314,
        0.72603561],
       [0.09774403, 0.09451231, 0.08853515, ..., 0.73475314, 1.        ,
        0.68109241],
       [0.12618273, 0.12534868, 0.11655191, ..., 0.72603561, 0.68109241,
        1.        ]])

In [ ]:
# TODO: Fix the distance, plot it in gelphi